In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('../input/movielens-100k-dataset/ml-100k/u.data', sep='\t',header=None).sample(frac=1)
print(df.head())
num_users = len(pd.unique(df[0]))
num_movie = len(pd.unique(df[1]))

X_train = df.drop(columns=[2,3])[:90000].to_numpy()
X_train = X_train - 1
y_train = df[2][:90000].to_numpy()
X_test = df.drop(columns=[2,3])[90000:].to_numpy()
y_test = df[2][90000:].to_numpy()

print(X_train.shape,y_train.shape)


         0     1  2          3
3493   108    13  3  879879834
71965  717   888  5  884642133
95786  709   693  4  879847082
59206  385  1022  3  883791570
27525  454   686  2  888267280
(90000, 2) (90000,)


In [48]:
import tensorflow as tf

def collaborative_model(num_users, num_movies, emb_dim):
    x = tf.keras.Input((2))
    x_user = tf.keras.layers.Embedding(num_users,emb_dim,name='users')(x[:,0])
    x_movie = tf.keras.layers.Embedding(num_movies,emb_dim,name='movies')(x[:,1])
    out = tf.linalg.diag_part(tf.linalg.matmul(x_user,x_movie,transpose_b=True))
    
    return tf.keras.Model(inputs=x, outputs=out, name="colab_model")

model = collaborative_model(num_users,num_movie,8)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse'])
model.fit(X_train,y_train,32,40,validation_split=0.2,
callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)])    
    

Epoch 1/40
2250/2250 [==============================] - 4s 1ms/step - loss: 11.0479 - mse: 11.0479 - val_loss: 5.0337 - val_mse: 5.0346
Epoch 2/40
2250/2250 [==============================] - 3s 1ms/step - loss: 2.6542 - mse: 2.6542 - val_loss: 1.6617 - val_mse: 1.6618
Epoch 3/40
2250/2250 [==============================] - 3s 1ms/step - loss: 1.3159 - mse: 1.3159 - val_loss: 1.1671 - val_mse: 1.1670
Epoch 4/40
2250/2250 [==============================] - 3s 1ms/step - loss: 1.0405 - mse: 1.0405 - val_loss: 1.0207 - val_mse: 1.0205
Epoch 5/40
2250/2250 [==============================] - 3s 1ms/step - loss: 0.9527 - mse: 0.9527 - val_loss: 0.9690 - val_mse: 0.9687
Epoch 6/40
2250/2250 [==============================] - 3s 1ms/step - loss: 0.9172 - mse: 0.9172 - val_loss: 0.9473 - val_mse: 0.9470
Epoch 7/40
2250/2250 [==============================] - 3s 2ms/step - loss: 0.9001 - mse: 0.9001 - val_loss: 0.9333 - val_mse: 0.9330
Epoch 8/40
2250/2250 [==============================] - 3s 2

In [49]:
items = pd.read_csv('../input/movielens-100k-dataset/ml-100k/u.item', sep='|',header=None,encoding='latin')
items

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from sklearn.manifold import TSNE
tsne = TSNE()
users = tsne.fit_transform(model.get_layer('users').get_weights()[0])

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [50]:
movies = tsne.fit_transform(model.get_layer('movies').get_weights()[0])

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [72]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# print(users.shape)
# plt.scatter(users[:,0], users[:,1])
import plotly.express as px
new_df = pd.DataFrame(movies)
new_df[2] = items[1].to_numpy()
new_df

fig = px.scatter(new_df, x=0, y=1,hover_data=[2])
fig.show()

,0,1,2
0,-18.552263,-18.599083,Toy Story (1995)
1,-29.939905,-8.408701,GoldenEye (1995)
2,-20.888079,5.382981,Four Rooms (1995)
3,28.661406,-37.155983,Get Shorty (1995)
4,-23.068695,-5.697330,Copycat (1995)
...,...,...,...
1677,-20.937172,51.264198,Mat' i syn (1997)
1678,-12.052010,2.261421,B. Monkey (1998)
1679,-17.368921,38.765377,Sliding Doors (1998)
1680,-1.903535,-16.012310,You So Crazy (1994)
